In [1]:
from utils import *
from pathlib import Path

%load_ext autoreload
%autoreload 2

/usr/local/lib/python3.10/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
dir_path = "papers"
arxiv_ids = [2210.03162, 2206.07682, 2306.08055]  #, 1904.07094, 2203.00759, 2104.08691, 2212.01681]

In [3]:
#text_file_path, abstract_file_path = create_corpus(dir_path, arxiv_ids)
text_file_path, abstract_file_path ='data/papers_text.txt', 'data/papers_abstract.txt'

In [4]:
dataset = create_dataset(text_file_path, abstract_file_path)

In [5]:
dataset['train']['text']

['Prompt Compression and Contrastive Conditioning for Controllability and Toxicity Reduction in Language Models : Introduction Language models (LMs), such as GPT-2 (Radford et al., 2018, 2019a), BERT (Devlin et al., 2018), T5 (Raffel et al., 2020), or GPT-3 (Brown et al., 2020), exhibit a remarkable ability to capture pat- terns of grammar, vocabulary, cultural knowledge, and conversational rhythms present in natural lan- guage. Formally, a LM is a conditional distribution over tokens p(xt|x1, · · · , xt−1), with each token xt ∈ V for some vocabulary V. Throughout this paper, we will refer to xh = x1, · · · , xt−1 as the prompt. This paper explores prompt compression: the idea that the text xh used to condition a LM can be approximately represented by a much smaller set of carefully chosen weights, using the framework of soft prompts (Lester et al., 2021). We begin by establishing some basic properties of compressed prompts, and importantly show that while highly ∗ Work done while at N

In [64]:
dataset['train']['text'][20]

'Tune As You Scale: Hyperparameter Optimization For Compute Efficient Training : Hyperparameter tuning of deep learning models can lead to order-of-magnitude performance gains for the same amount of compute. Despite this, sys- tematic tuning is uncommon, particularly for large models, which are expensive to evaluate and tend to have many hyperparameters, necessitating dif- ficult judgment calls about tradeoffs, budgets, and search bounds. To address these issues and pro- pose a practical method for robustly tuning large models, we present Cost-Aware Pareto Region Bayesian Search (CARBS), a Bayesian optimiza- tion algorithm that performs local search around the performance-cost Pareto frontier. CARBS does well even in unbounded search spaces with many hyperparameters, learns scaling relation- ships so that it can tune models even as they are scaled up, and automates much of the “black magic” of tuning. Among our results, we effec- tively solve the entire ProcGen benchmark just by tuning

In [19]:
text = ''
lines = extract_text_from_pdf(pdf_path).splitlines()
lines = [line for line in lines if len(line) >= 4]
for i in range(len(lines)):
    if lines[i].lower() == 'abstract':
        abstract_idx = i + 1
    if lines[i].lower() == 'introduction':
        intro_idx = i
        abstract = " ".join(lines[abstract_idx:intro_idx])
        text =  " ".join(lines[intro_idx:])
    #return abstract, text

In [21]:
text[:200]

'Introduction Language models (LMs), such as GPT-2 (Radford et al., 2018, 2019a), BERT (Devlin et al., 2018), T5 (Raffel et al., 2020), or GPT-3 (Brown et al., 2020), exhibit a remarkable ability to ca'

In [ ]:
def clean_pdf_lines(pdf_path, str_len):
    # remove  lines with string length < str_len
    # remove lines before theabstract
    # return lines
    lines = extract_text_from_pdf(pdf_path).splitlines()
    lines = [line for line in lines if len(line) >= str_len]
    for i in range(len(lines)):
        if lines[i].lower() == 'abstract':
            return " ".join(lines[i:])
    warnings.warn("Warning: Did not parse abstract of paper", pdf_path)
    return []

In [7]:
train, val = create_dataset(dir_path, arxiv_ids)


 Created train_corpus.txt and val_corpus.txt

 Created train and val splits


In [25]:
train.split

'train'

In [ ]:
#tokenize
import numpy as np
import tiktoken

def tokenize(dataset):
     
    split =  dataset.split
    enc = tiktoken.get_encoding("gpt2")
    prefix = "abstract: "
    def process(example):
        #based both on karpathy's prepare.py and  https://huggingface.co/docs/transformers/tasks/summarization
        example_input = [prefix + doc for doc in example["text"]]
        ids = enc.encode_ordinary(example_input) # encode_ordinary ignores any special tokens
        ids.append(enc.eot_token) # add the end of text token, e.g. 50256 for gpt2 bpe
        # note: check if eot should be prepended not appended 
        label = enc.encode_ordinary(example["abstract"])
        #out = {'ids': ids, 'len': len(ids)}
        return {'ids': ids, 'label': label, 'len': len(ids)}

    tokenized = dataset.map( #https://github.com/huggingface/datasets/blob/src/datasets/dataset_dict.py
            process,
            remove_columns=['text'],
            desc="tokenizing the splits"
        )

    for split, dset in tokenized.items():
            arr_len = np.sum(dset['len'], dtype=np.uint64)
            filename = os.path.join(os.path.dirname(__file__), f'{split}.bin')
            dtype = np.uint16 # (can do since enc.max_token_value == 50256 is < 2**16)
            arr = np.memmap(filename, dtype=dtype, mode='w+', shape=(arr_len,))
            total_batches = 1024

            idx = 0
            for batch_idx in tqdm(range(total_batches), desc=f'writing {filename}'):
                # Batch together samples for faster write
                batch = dset.shard(num_shards=total_batches, index=batch_idx, contiguous=True).with_format('numpy')
                arr_batch = np.concatenate(batch['ids'])
                # Write into mmap
                arr[idx : idx + len(arr_batch)] = arr_batch
                idx += len(arr_batch)
            arr.flush()

In [ ]:
prefix = "abstract: "
def preprocess_function(examples):
    #based on https://huggingface.co/docs/transformers/tasks/summarization
    inputs = [prefix + doc for doc in examples["text"]]
    model_inputs = tokenizer(inputs, max_length=1024, truncation=True)

    labels = tokenizer(text_target=examples["abstract"], max_length=128, truncation=True)

    model_inputs["labels"] = labels["input_ids"]
    return model_inputs

tokenized_billsum = train_split.map(preprocess_function, batched=True)

In [ ]:
remove references, use parts of the article to predict other parts, e.g. predict the abstract and conclusion

or

In [23]:
from pathlib import Path

from py_pdf_parser.loaders import load_file


import pdfminer
from pdfminer.high_level import extract_text
#%load_ext autoreload
#%autoreload 2

#import sys
#print(sys.path)

In [69]:
import os
from urllib.request import urlretrieve
import bibtexparser

dir_path = "papers"

arxiv_id = "2210.03162"
arxiv_id_bib = "2210-03162"
pdf_url = f"https://export.arxiv.org/pdf/{arxiv_id}"
bibtex_url = f"https://dblp.uni-trier.de/rec/journals/corr/abs-{arxiv_id_bib}.bib?param=1"

pdf_local_path = os.path.join(dir_path, "test.pdf")
bibtex_local_path = os.path.join(dir_path, "test.bib")

if pdf_url is not None:
    pdf_path, headers = urlretrieve(pdf_url, pdf_local_path)

if bibtex_url is not None:
    bib_path, headers = urlretrieve(bibtex_url, bibtex_local_path)


In [ ]:
def download_bibtex(arxiv_id, dir_path):

    arxiv_id_bib = str(arxiv_id).replace('.', '-')
    bibtex_url = f"https://dblp.uni-trier.de/rec/journals/corr/abs-{arxiv_id_bib}.bib?param=1"

    bibtex_local_path = os.path.join(dir_path, f"{arxiv_id_bib}.bib")

    if bibtex_url is not None:
        bib_path, _ = urlretrieve(bibtex_url, bibtex_local_path)

    return bib_path

In [121]:

def download_paper(arxiv_id, dir_path):

    pdf_url = f"https://export.arxiv.org/pdf/{arxiv_id}"

    #arxiv_id =  str(arxiv_id).replace('.', '-')
    pdf_local_path = os.path.join(dir_path, f"{arxiv_id}.pdf")

    if pdf_url is not None:
        pdf_path, _ = urlretrieve(pdf_url, pdf_local_path)

    return pdf_path



In [122]:
download_paper(2210.03162, dir_path)

'papers/2210.03162.pdf'

In [117]:
import re

def extract_metadata(bib_path):
    library = bibtexparser.parse_file(bib_path) 

    authors = library.entries[0].fields_dict['author']._value.splitlines()
    authors = [re.sub("\s{2,}"," ",elem.replace('and ','')) for elem in authors] 

    title = library.entries[0].fields_dict['title']._value
    title = re.sub("\s{2,}"," ",title)
    return title, authors

extract_metadata(bib_path)

('Prompt Compression and Contrastive Conditioning for Controllability and Toxicity Reduction in Language Models',
 ['David Wingate and', ' Mohammad Shoeybi and', ' Taylor Sorensen'])

In [99]:
title.replace('\n','')

'Prompt Compression and Contrastive Conditioning for Controllability                  and Toxicity Reduction in Language Models'

In [ ]:
from urllib.request import urlretrieve

path, headers = urlretrieve(url, filename)
for name, value in headers.items():
    print(name, value)

In [25]:
dir_path =  "papers"

def list_pdfs(dir_path):
     pathlist = Path(dir_path).glob('**/*.pdf')
     titles = []
     for path in pathlist:
          titles.append(str(path))

     return titles

def extract_text(file_path):
     return extract_text(file_path)


In [26]:
list_pdfs(dir_path)

['papers/Emergent_Abilities_of_Large_Language_Models.pdf',
 'papers/Prompt_Programming_for_Large_Language_Models_Beyond_the_Few-Shot_Paradigm.pdf']

In [13]:
text = extract_text('papers/Emergent_Abilities_of_Large_Language_Models.pdf')


In [51]:
print(text[40:350].splitlines())


['', '8', '6', '7', '0', '.', '6', '0', '2', '2', ':', 'v', 'i', 'X', 'r', 'a', '', 'Published in Transactions on Machine Learning Research (08/2022)', '', 'Emergent Abilities of Large Language Models', '', 'Jason Wei 1', 'Yi Tay 1', 'Rishi Bommasani 2', 'Colin Raﬀel 3', 'Barret Zoph 1', 'Sebastian Borgeaud 4', 'Dani Yogatama 4', 'Maarten Bosma 1', 'Denny Zhou 1', 'Donald Metzler 1', 'Ed H. Chi 1', 'Tatsu']


In [32]:
from io import StringIO

from pdfminer.converter import TextConverter
from pdfminer.layout import LAParams
from pdfminer.pdfdocument import PDFDocument
from pdfminer.pdfinterp import PDFResourceManager, PDFPageInterpreter
from pdfminer.pdfpage import PDFPage
from pdfminer.pdfparser import PDFParser

output_string = StringIO()
with open('papers/Emergent_Abilities_of_Large_Language_Models.pdf', 'rb') as in_file:
    parser = PDFParser(in_file)
    doc = PDFDocument(parser)
    rsrcmgr = PDFResourceManager()
    device = TextConverter(rsrcmgr, output_string, laparams=LAParams())
    interpreter = PDFPageInterpreter(rsrcmgr, device)
    for page in PDFPage.create_pages(doc):
        interpreter.process_page(page)

print(output_string.getvalue()[40:80] == text[40:80])

True


In [3]:
list_pdfs(dir_path)

['papers/Emergent_Abilities_of_Large_Language_Models.pdf',
 'papers/Prompt_Programming_for_Large_Language_Models_Beyond_the_Few-Shot_Paradigm.pdf']

In [4]:
document =  load_pdf('papers/Emergent_Abilities_of_Large_Language_Models.pdf')

In [11]:
document

In [9]:
abstract = document.elements.filter_by_text_equal("2").extract_single_element()
abstract

<PDFElement tags: set(), font: 'YBSOMS+LMRoman10-Regular,10.0'>

In [10]:
document.elements.to_the_right_of(abstract).extract_single_element().text()

NoElementFoundError: There are no elements in the ElementList